# Fitting `lmfit.Model`s to `xarray` objects

`xarray-lmfit` adds some methods to xarray objects that allows you to fit data with lmfit models: {meth}`xarray.DataArray.xlm.modelfit` and {meth}`xarray.Dataset.xlm.modelfit`, depending on whether you want to fit a single DataArray or multiple DataArrays in a Dataset.

:::{hint}

The syntax of the accessors are similar to the xarray native methods {meth}`xarray.DataArray.curvefit` and {meth}`xarray.Dataset.curvefit`.

:::

First, let us generate a Gaussian peak on a linear background.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import lmfit

# Generate Gaussian peak on linear background
x = np.linspace(0, 10, 50)
y = -0.1 * x + 2 + 3 * np.exp(-((x - 5) ** 2) / (2 * 1**2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
yerr = np.full_like(x, 0.3)
y = rng.normal(y, yerr)

y_arr = xr.DataArray(y, dims=("x",), coords={"x": x})

y_arr

Here, `y_arr` is the DataArray that contains both the values we want to fit, along with the independent variable `x` as a coordinate.

After importing `xarray_lmfit`, we can use {meth}`xarray.DataArray.xlm.modelfit` to fit the data to a model.

In [ ]:
import xarray_lmfit

model = lmfit.models.GaussianModel() + lmfit.models.LinearModel()

params = model.make_params(slope=-0.1, center=5.0, sigma={"value": 0.1, "min": 0})

result = y_arr.xlm.modelfit("x", model=model, params=params)

## The fit result Dataset

{meth}`xarray.DataArray.xlm.modelfit` returns a {class}`xarray.Dataset` including the best-fit parameters and the fit statistics:

In [ ]:
result

Let's take a closer look at the data variables in the resulting Dataset.

- `modelfit_results` contains the underlying {class}`lmfit.model.ModelResult` object from the fit.
- `modelfit_coefficients` and `modelfit_stderr` contain the best-fit coefficients and their errors, respectively.
- `modelfit_stats` contains the [goodness-of-fit statistics](https://lmfit.github.io/lmfit-py/fitting.html#goodness-of-fit-statistics).

When called on a Dataset instead of a DataArray, these variables will be prefixed with the name of the data variable they correspond to.

It may not be immediately obvious how this is useful, but the true power of the accessor comes from its ability to utilize xarray's powerful broadcasting capabilities, as described in the next section.